In [1]:
import pandas as pd
import pickle
import sqlite3
import re
import sys
sys.path.append('..\\src')
import sqllite_database


In [10]:
# UNPICKLE CITYBIKES DF
with open('../data/citybikes_filled_df.pickle', 'rb') as df:
    citybikes_df = pickle.load(df)

citybikes_df.head(5)

,name,id,timestamp,longitude,latitude,slots,free_bikes,empty_slots,has_ebikes,ebikes,payment,renting,returning
0,Harmon St at Adeline St,d0e8f4f1834b7b33a3faf8882f567ab8,2024-04-02T18:29:27.066000Z,-122.270582,37.849735,15,9,6,True,0,"[key, creditcard]",1,1
1,Fountain Alley at S 2nd St,983514094dd808b1604da2dcfc2d09af,2024-04-02T18:29:26.582000Z,-121.889277,37.336188,11,8,3,True,6,"[key, creditcard]",1,1
2,Oak St at 1st St,da17603652106fda93da4e255a5b0a22,2024-04-02T18:29:27.044000Z,-121.88109,37.322125,23,14,9,True,7,"[key, creditcard]",1,1
3,Bestor Art Park,7a21c92b3b4cd2f7759107b4fdebf869,2024-04-02T18:29:26.797000Z,-121.874119,37.323678,15,12,3,True,3,"[key, creditcard]",1,1
4,5th St at Virginia St,ce34d38fb230a23c1ced12d1e16df294,2024-04-02T18:29:26.971000Z,-121.87712,37.325998,27,19,7,True,12,"[key, creditcard]",1,1


In [36]:
non_unique_values = citybikes_df[citybikes_df.duplicated('id', keep=False)]['id'].unique()
non_unique_values

array([], dtype=object)

In [27]:
# UNPICKLE FSQ DF 
with open('../data/fsq_places.pickle', 'rb') as df:
    fsq_places = pickle.load(df)

fsq_places.head(5)

,reference_bike_stn,fsq_id,cat_id,cat_name,lat,long,name,street_address,zip,locality,distance
0,d0e8f4f1834b7b33a3faf8882f567ab8,4b3fc82cf964a520a1ae25e3,13303,Mexican Restaurant,37.850052,-122.270523,Las Palmas Super Burritos and Heros,3221 Adeline St,94703,Berkeley,33
1,d0e8f4f1834b7b33a3faf8882f567ab8,57da76a603a44d3f9b3ddd3b,10004,Art Gallery,37.849798,-122.271728,Firehouse Collective East/West Art Studios,1835 Harmon St,94703,Berkeley,100
2,d0e8f4f1834b7b33a3faf8882f567ab8,4b64bd80f964a52074cc2ae3,13064,Pizzeria,37.849316,-122.271629,Domino's Pizza,3264 Adeline St,94703,Berkeley,113
3,d0e8f4f1834b7b33a3faf8882f567ab8,4c12e9a51b5cef3b9cd7edc4,13102,Cantonese Restaurant,37.848703,-122.270660,Chen's Garden,1818 Alcatraz Ave,94703,Berkeley,119
4,d0e8f4f1834b7b33a3faf8882f567ab8,52fe4532498e80c8d231d768,13034,Café,37.850958,-122.271077,Rasa Caffe,3140 Martin Luther King Jr Way,94703,Berkeley,141


In [45]:
# UNPICKLE YELP DF 
with open('../data/yelp_df.pickle', 'rb') as df:
    yelp_df = pickle.load(df)

yelp_df.head(5)

,reference_bike_stn,yelp_id,cat_alias,lat,long,name,street_address,zip,city,price,rating,review_count,distance_from_bike_stn
0,d0e8f4f1834b7b33a3faf8882f567ab8,OjMumhyL26n5fDZ2CQLnyg,ethiopian,37.850301,-122.271266,Lemat,3212 Adeline St,94703,Berkeley,$$,4.4,239,86.993559
1,d0e8f4f1834b7b33a3faf8882f567ab8,-EpmBwODsvQ6gHx2IuVO5w,cafes,37.850017,-122.270532,El Tiny Cafe,3219 Adeline St,94703,Berkeley,None,4.8,60,31.707767
2,d0e8f4f1834b7b33a3faf8882f567ab8,shRP4Vxu2hDdmdsHf-5DmQ,burgers,37.848520,-122.260380,A+ Burger,6228 Telegraph Ave,94609,Oakland,$$,4.4,859,895.542416
3,d0e8f4f1834b7b33a3faf8882f567ab8,pJdwr6JoCVXTKbdJlm-rdg,italian,37.850900,-122.278200,Creekwood,3121 Sacramento St,94702,Berkeley,$$,4.6,481,684.391900
4,d0e8f4f1834b7b33a3faf8882f567ab8,nt46Amrqx5qwTyNEcO54ew,burgers,37.843820,-122.276750,Sideshow Kitchen,942 Stanford Ave,94608,Oakland,$$,4.4,496,853.805062


# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [3]:
con = sqlite3.connect("../data/biking_bayarea.db")

In [4]:
cur = con.cursor() # create database cursor, used to execute SQL queries & fetch results

In [5]:
# cur.execute("""
# CREATE TABLE citybikes (
#     name VARCHAR(50),
#     id VARCHAR(100) PRIMARY KEY NOT NULL,
#     timestamp TIMESTAMP,
#     longitude VARCHAR(50),
#     latitude VARCHAR(50),
#     slots INT UNSIGNED,
#     free_bikes INT UNSIGNED,
#     empty_slots INT UNSIGNED,
#     has_ebikes BOOLEAN,
#     ebikes INT UNSIGNED,
#     payment VARCHAR(50),
#     renting INT UNSIGNED,
#     is_returning INT UNSIGNED
# );
# """)

OperationalError: table citybikes already exists

In [13]:
# sqllite_database.populate_citybikes_sqllite(con, cur, citybikes_df)

In [14]:
cur.execute("SELECT COUNT(*) FROM citybikes;")
row = cur.fetchone()
print(row[0])

555


In [73]:
cur.execute("SELECT * FROM citybikes LIMIT 5;")
rows = cur.fetchall()
for row in rows:
    print(row)

('Harmon St at Adeline St', 'd0e8f4f1834b7b33a3faf8882f567ab8', '2024-04-02T18:29:27.066000Z', '-122.270582', '37.849735', 15, 9, 6, 1, 0, 1, 1)
('Fountain Alley at S 2nd St', '983514094dd808b1604da2dcfc2d09af', '2024-04-02T18:29:26.582000Z', '-121.889276504517', '37.3361883002906', 11, 8, 3, 1, 6, 1, 1)
('Oak St at 1st St', 'da17603652106fda93da4e255a5b0a22', '2024-04-02T18:29:27.044000Z', '-121.881090402603', '37.3221246254486', 23, 14, 9, 1, 7, 1, 1)
('Bestor Art Park', '7a21c92b3b4cd2f7759107b4fdebf869', '2024-04-02T18:29:26.797000Z', '-121.8741186', '37.3236779', 15, 12, 3, 1, 3, 1, 1)
('5th St at Virginia St', 'ce34d38fb230a23c1ced12d1e16df294', '2024-04-02T18:29:26.971000Z', '-121.87712', '37.3259984', 27, 19, 7, 1, 12, 1, 1)


In [79]:
cur.execute("PRAGMA table_info(citybikes);")
rows = cur.fetchall()
for row in rows:
    print(row)

(0, 'name', 'VARCHAR(50)', 0, None, 0)
(1, 'id', 'VARCHAR(100)', 1, None, 1)
(2, 'timestamp', 'TIMESTAMP', 0, None, 0)
(3, 'longitude', 'VARCHAR(50)', 0, None, 0)
(4, 'latitude', 'VARCHAR(50)', 0, None, 0)
(5, 'slots', 'INT UNSIGNED', 0, None, 0)
(6, 'free_bikes', 'INT UNSIGNED', 0, None, 0)
(7, 'empty_slots', 'INT UNSIGNED', 0, None, 0)
(8, 'has_ebikes', 'BOOLEAN', 0, None, 0)
(9, 'ebikes', 'INT UNSIGNED', 0, None, 0)
(10, 'renting', 'INT UNSIGNED', 0, None, 0)
(11, 'is_returning', 'INT UNSIGNED', 0, None, 0)


In [17]:
cur.execute("UPDATE citybikes SET payment = REPLACE(payment, '''', '')")
con.commit()

cur.execute("SELECT * FROM citybikes LIMIT 5;")
rows = cur.fetchall()
for row in rows:
    print(row)

In [14]:
# cur.execute("ALTER TABLE citybikes DROP COLUMN payment")
# con.commit()

In [18]:
# cur.execute("UPDATE citybikes SET name = TRIM(name), id = TRIM(id);")
# con.commit()

In [90]:
# cur.execute("""
# CREATE TABLE fsq (
#     reference_bike_stn VARCHAR(100),
#     fsq_id VARCHAR(100),
#     cat_id INT UNSIGNED,
#     cat_name VARCHAR(100),
#     lat SMALLINT UNSIGNED,
#     long SMALLINT,
#     name VARCHAR(100),
#     street_address VARCHAR(100),
#     zip INT UNSIGNED,
#     locality VARCHAR(50),
#     distance SMALLINT UNSIGNED,
#     PRIMARY KEY (reference_bike_stn, fsq_id),
#     FOREIGN KEY (reference_bike_stn) REFERENCES citybikes(id)
# );
# """)

In [91]:
# sqllite_database.populate_fsq_sqllite(con, cur, fsq_places)

In [92]:
cur.execute("SELECT COUNT(*) FROM fsq;")
row = cur.fetchone()
print(row[0])

25677


In [96]:
cur.execute("SELECT * FROM fsq LIMIT 5;")
rows = cur.fetchall()
for row in rows:
    print(row)

('d0e8f4f1834b7b33a3faf8882f567ab8', '4b3fc82cf964a520a1ae25e3', 13303, 'Mexican Restaurant', 37.850052, -122.270523, 'Las Palmas Super Burritos and Heros', '3221 Adeline St', 94703, 'Berkeley', 33)
('d0e8f4f1834b7b33a3faf8882f567ab8', '57da76a603a44d3f9b3ddd3b', 10004, 'Art Gallery', 37.849798, -122.271728, 'Firehouse Collective East/West Art Studios', '1835 Harmon St', 94703, 'Berkeley', 100)
('d0e8f4f1834b7b33a3faf8882f567ab8', '4b64bd80f964a52074cc2ae3', 13064, 'Pizzeria', 37.849316, -122.271629, "Domino's Pizza", '3264 Adeline St', 94703, 'Berkeley', 113)
('d0e8f4f1834b7b33a3faf8882f567ab8', '4c12e9a51b5cef3b9cd7edc4', 13102, 'Cantonese Restaurant', 37.848703, -122.27066, "Chen's Garden", '1818 Alcatraz Ave', 94703, 'Berkeley', 119)
('d0e8f4f1834b7b33a3faf8882f567ab8', '52fe4532498e80c8d231d768', 13034, 'Café', 37.850958, -122.271077, 'Rasa Caffe', '3140 Martin Luther King Jr Way', 94703, 'Berkeley', 141)


In [94]:
cur.execute("""
UPDATE fsq
SET name = TRIM(name),
    reference_bike_stn = TRIM(reference_bike_stn),
    fsq_id = TRIM(fsq_id),
    cat_name = TRIM(cat_name),
    locality = TRIM(locality);
""")
con.commit()


In [95]:
cur.execute("PRAGMA table_info(fsq);")
rows = cur.fetchall()
for row in rows:
    print(row)

(0, 'reference_bike_stn', 'VARCHAR(100)', 0, None, 1)
(1, 'fsq_id', 'VARCHAR(100)', 0, None, 2)
(2, 'cat_id', 'INT UNSIGNED', 0, None, 0)
(3, 'cat_name', 'VARCHAR(100)', 0, None, 0)
(4, 'lat', 'SMALLINT UNSIGNED', 0, None, 0)
(5, 'long', 'SMALLINT', 0, None, 0)
(6, 'name', 'VARCHAR(100)', 0, None, 0)
(7, 'street_address', 'VARCHAR(100)', 0, None, 0)
(8, 'zip', 'INT UNSIGNED', 0, None, 0)
(9, 'locality', 'VARCHAR(50)', 0, None, 0)
(10, 'distance', 'SMALLINT UNSIGNED', 0, None, 0)


In [40]:
# cur.execute("ALTER TABLE fsq DROP COLUMN cat_id;")
# con.commit()

In [56]:
# cur.execute("""
# CREATE TABLE yelp (
#     reference_bike_stn VARCHAR(100),	
#     yelp_id	varchar(100),
#     cat_alias VARCHAR(40),
#     lat	INT UNSIGNED,
#     long INT, 
#     name VARCHAR(50),
#     street_address VARCHAR(100),	
#     zip	INT UNSIGNED,
#     city VARCHAR(50),
#     price VARCHAR(5),
#     rating FLOAT UNSIGNED,
#     review_count INT UNSIGNED,
#     distance_from_bike_stn FLOAT UNSIGNED,
#     PRIMARY KEY (reference_bike_stn, yelp_id)
# );
# """)

In [57]:
# sqllite_database.populate_yelp_sqllite(con, cur, yelp_df)

In [58]:
cur.execute("SELECT COUNT(*) FROM yelp;")
row = cur.fetchone()
print(row[0])

27315


In [68]:
cur.execute("SELECT * FROM yelp LIMIT 5;")
rows = cur.fetchall()
for row in rows:
    print(row)

('d0e8f4f1834b7b33a3faf8882f567ab8', 'OjMumhyL26n5fDZ2CQLnyg', 'ethiopian', 37.850300662973346, -122.27126646102298, 'Lemat', '3212 Adeline St', 94703, 'Berkeley', '$$', 4.4, 239, 87)
('d0e8f4f1834b7b33a3faf8882f567ab8', '-EpmBwODsvQ6gHx2IuVO5w', 'cafes', 37.85001740687401, -122.27053198326358, 'El Tiny Cafe', '3219 Adeline St', 94703, 'Berkeley', None, 4.8, 60, 32)
('d0e8f4f1834b7b33a3faf8882f567ab8', 'shRP4Vxu2hDdmdsHf-5DmQ', 'burgers', 37.84852, -122.26038, 'A+ Burger', '6228 Telegraph Ave', 94609, 'Oakland', '$$', 4.4, 859, 896)
('d0e8f4f1834b7b33a3faf8882f567ab8', 'pJdwr6JoCVXTKbdJlm-rdg', 'italian', 37.8509, -122.2782, 'Creekwood', '3121 Sacramento St', 94702, 'Berkeley', '$$', 4.6, 481, 684)
('d0e8f4f1834b7b33a3faf8882f567ab8', 'nt46Amrqx5qwTyNEcO54ew', 'burgers', 37.84382, -122.27675, 'Sideshow Kitchen', '942 Stanford Ave', 94608, 'Oakland', '$$', 4.4, 496, 854)


In [59]:
cur.execute("UPDATE yelp SET yelp_id = TRIM(yelp_id), cat_alias = TRIM(cat_alias), name = TRIM(name), street_address = TRIM(street_address);")
con.commit()

In [67]:
cur.execute("""
CREATE TABLE new_yelp AS
SELECT reference_bike_stn, yelp_id, cat_alias, lat, long, name, street_address, zip, city, price, rating, review_count, CAST(distance_from_bike_stn AS INTEGER) AS distance_from_bike_stn
FROM yelp;
""")
cur.execute("DROP TABLE yelp;")
cur.execute("ALTER TABLE new_yelp RENAME TO yelp;")
con.commit()

In [86]:
cur.execute("""
CREATE TABLE temp_yelp (
    reference_bike_stn TEXT,
    yelp_id TEXT,
    cat_alias TEXT,
    lat INT,
    long INT,
    name TEXT,
    street_address TEXT,
    zip INT,
    city TEXT,
    price TEXT,
    rating REAL,
    review_count INT,
    distance_from_bike_stn INT,
    PRIMARY KEY (reference_bike_stn, yelp_id),
    FOREIGN KEY (reference_bike_stn) REFERENCES citybikes(id)
);
""")

cur.execute("INSERT INTO temp_yelp SELECT * FROM yelp;")
cur.execute("DROP TABLE yelp;")
cur.execute("ALTER TABLE temp_yelp RENAME TO yelp;")
con.commit()

In [87]:
cur.execute("PRAGMA table_info(yelp);")
rows = cur.fetchall()
for row in rows:
    print(row)

(0, 'reference_bike_stn', 'TEXT', 0, None, 1)
(1, 'yelp_id', 'TEXT', 0, None, 2)
(2, 'cat_alias', 'TEXT', 0, None, 0)
(3, 'lat', 'INT', 0, None, 0)
(4, 'long', 'INT', 0, None, 0)
(5, 'name', 'TEXT', 0, None, 0)
(6, 'street_address', 'TEXT', 0, None, 0)
(7, 'zip', 'INT', 0, None, 0)
(8, 'city', 'TEXT', 0, None, 0)
(9, 'price', 'TEXT', 0, None, 0)
(10, 'rating', 'REAL', 0, None, 0)
(11, 'review_count', 'INT', 0, None, 0)
(12, 'distance_from_bike_stn', 'INT', 0, None, 0)


Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

Look at the data before and after the join to validate your data.